In [3]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

Why we use sqrt because it help to keep varince =1

In [1]:
import numpy as np

# Function to compute variance before and after scaling
def compute_variance(dim, num_trials=1000):
    dot_products = []
    scaled_dot_products = []

    # Generate multiple random vectors and compute dot products
    for _ in range(num_trials):
        q = np.random.randn(dim)
        k = np.random.randn(dim)

        # Compute dot product
        dot_product = np.dot(q, k)
        dot_products.append(dot_product)

        # Scale the dot product by sqrt(dim)
        scaled_dot_product = dot_product / np.sqrt(dim)
        scaled_dot_products.append(scaled_dot_product)

    # Calculate variance of the dot products
    variance_before_scaling = np.var(dot_products)
    variance_after_scaling = np.var(scaled_dot_products)

    return variance_before_scaling, variance_after_scaling

# For dimension 5
variance_before_5, variance_after_5 = compute_variance(5)
print(f"Variance before scaling (dim=5): {variance_before_5}")
print(f"Variance after scaling (dim=5): {variance_after_5}")

# For dimension 20
variance_before_100, variance_after_100 = compute_variance(100)
print(f"Variance before scaling (dim=100): {variance_before_100}")
print(f"Variance after scaling (dim=100): {variance_after_100}")



Variance before scaling (dim=5): 5.569993232122386
Variance after scaling (dim=5): 1.113998646424477
Variance before scaling (dim=100): 89.35346930640613
Variance after scaling (dim=100): 0.8935346930640611


In [10]:
import torch.nn as nn
class self_attentionv1(nn.Module):
  def __init__(self,d_in,d_out):# input and output dimension
    super().__init__()
    self.w_query=nn.Parameter(torch.rand(d_in,d_out))
    self.w_key=nn.Parameter(torch.rand(d_in,d_out))
    self.w_value=nn.Parameter(torch.rand(d_in,d_out))

  def forward(self,x):
    key = x @ self.w_key
    query=x @ self.w_query
    value= x @ self.w_value

    attention_score= query @ key.T
    attention_weight= torch.softmax( attention_score / key.shape[-1]**0.5 ,dim =-1)
    contex_vector =attention_weight @ value
    return contex_vector


In [13]:
torch.manual_seed(123)
d_in = inputs.shape[1]
d_out = 2
sa_v1 = self_attentionv1(d_in, d_out)
print(sa_v1.forward(inputs))
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)
tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


Additionally, a significant advantage of using nn.Linear instead of manually
implementing nn.Parameter(torch.rand(...)) is that nn.Linear has an optimized weight
initialization scheme, contributing to more stable and effective model training.

meaning of self.w_key(x)
y=(x)*(W)+b


In [24]:
import torch.nn as nn
class self_attentionv2(nn.Module):
  def __init__(self,d_in,d_out,qkv_bias=False):# input and output dimension
    super().__init__()
    self.w_query=nn.Linear(d_in,d_out,bias= qkv_bias)
    self.w_key=nn.Linear(d_in,d_out,bias= qkv_bias)
    self.w_value=nn.Linear(d_in,d_out,bias= qkv_bias)

  def forward(self,x):
    key = self.w_key(x)
    query=self.w_query(x)
    value= self.w_value(x)

    attention_score= query @ key.T
    attention_weight= torch.softmax( attention_score / key.shape[-1]**0.5 ,dim =-1)
    contex_vector =attention_weight @ value
    return contex_vector


In [25]:
torch.manual_seed(123)
d_in = inputs.shape[1]
d_out = 2
sa_v2 = self_attentionv2(d_in, d_out)
print(sa_v2.forward(inputs))
print(sa_v2(inputs))

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)
tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)
